In [ ]:
import numpy as np
from lsst.daf.butler import Butler, DatasetNotFoundError
import matplotlib.animation as animation
from IPython.display import HTML

from lsst.obs.lsst import LsstCam
import matplotlib.pyplot as plt
%matplotlib widget

In [ ]:
butler = Butler("LSSTCam", collections=["LSSTCam/raw/guider"], instrument="LSSTCam")

In [ ]:
dets = {
    "R00_SG0": (189, 3),
    "R00_SG1": (190, 2),
    "R04_SG0": (193, 0),
    "R04_SG1": (194, 3),
    "R40_SG0": (197, 2),
    "R40_SG1": (198, 1),
    "R44_SG0": (201, 1),
    "R44_SG1": (202, 0),
}

drs = butler.registry.queryDatasets("guider_raw", where="detector=189")
for exposure in range(2025050200267, 2025050200268):
# for dr in drs:
    # exposure = dr.dataId["exposure"]
    print(exposure)
    stamps = {}
    arrs = {}
    try:
        for k, (det, nRot90) in dets.items():
            raw = butler.get("guider_raw", exposure=exposure, detector=det)
            stamps[k] = raw
            for i, stamp in enumerate(stamps[k]):
                if i == 0:
                    arrs[k] = np.full((len(stamps[k]), 400, 400), np.nan, dtype=float)
                arr = stamp.stamp_im.image.array
                # Note: Tested on ROI segments starting with 0.  
                # Might need to rot180 if the ROI segment starts with a 1.
                arrs[k][i] = np.rot90(arr, nRot90)
    except DatasetNotFoundError:
        continue
    
    norm_full = False
    
    # Normalize between 2% and 98%
    for k, arr in arrs.items():
        arr[:] -= np.nanmedian(arr)
    
    if norm_full:
        vmin, vmax = np.nanquantile(
            np.concatenate(
                [arr for arr in arrs.values()],  
            ),
            [0.02, 0.98]
        )
    
    for k, arr in arrs.items():
        if not norm_full:
            vmin, vmax = np.nanquantile(arr, [0.02, 0.98])
        arr[:] = arr - vmin
        arr[:] = arr / (vmax-vmin)    

    layout = [
        [".", "R40_SG1", "R44_SG0", "."],
        ["R40_SG0", ".", ".", "R44_SG1"],
        ["R00_SG1", ".", ".", "R04_SG0"],
        [".", "R00_SG0", "R04_SG1", "."],
    ]
    fig, axs = plt.subplot_mosaic(layout, figsize=(5, 5), )
    images = {}
    for key, ax in axs.items():
        ax.set_aspect('equal')
        ax.set_xticks([])
        ax.set_yticks([])
        ax.text(0.05, 0.9, key, transform=ax.transAxes, fontsize=6, color="w")
        images[key] = ax.imshow(np.zeros((400, 400)), vmin=0, vmax=1, origin="lower", cmap="Grays_r")
            
    plt.suptitle(str(exposure), fontsize=9)
    plt.tight_layout()        

    def update(frame):
        out = []
        for k, im in images.items():
            if frame >= len(stamps[k]):
                im.set_array(np.zeros((400, 400), dtype=float))
                continue
            arr = stamps[k][frame].stamp_im.image.array
            im.set_array(np.transpose(arrs[k][frame]))
            out.append(im)
        return out
    
    # 50 frames in 30 sec = 600ms per frame.  Show at 10x speed so 60ms per frame.
    ani = animation.FuncAnimation(fig=fig, func=update, frames=max(len(v) for v in stamps.values()), interval=60)
    ani.save(f"guide_{exposure}.mp4", fps=30, writer='ffmpeg')